<a href="https://colab.research.google.com/github/ShamsRupak/ai-doc-processing-suite/blob/main/Experimenting_with_Model_Performance_TinyLlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -U llama-index
!pip install -U llama-index-llms-huggingface
!pip install -U llama-index-embeddings-huggingface
!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q gradio
!pip install einops accelerate bitsandbytes
!pip install PyMuPDF

In [7]:
import fitz  # PyMuPDF

# Load and extract text from PDF
pdf_path = "sample_contract.pdf"
doc = fitz.open(pdf_path)
text = "\n".join([page.get_text() for page in doc])
print(f"Extracted {len(text.split())} words.")

Extracted 315 words.


In [12]:
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts import PromptTemplate
import torch

# Set prompt formats
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

# Load TinyLlama
llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.bfloat16}
)

In [15]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Document, VectorStoreIndex
from llama_index.core import Settings

# Load embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Create document
documents = [Document(text=text)]

# Configure Settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024

# Build index
index = VectorStoreIndex.from_documents(documents)

# Create query engine
query_engine = index.as_query_engine()

In [17]:
# Build index
index = VectorStoreIndex.from_documents(documents)

# Create query engine
query_engine = index.as_query_engine()

# Try suggested queries
queries = [
    "What are the penalties for late payments?",
    "Summarize the key terms in this contract.",
    "What is the refund policy?"
]

# Run each query
for q in queries:
    print(f"\n--- Query: {q} ---")
    response = query_engine.query(q)
    print(response)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- Query: What are the penalties for late payments? ---


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



The penalties for late payments in this Agreement are as follows:

1. Late payments shall bear interest at the rate of 1.5% per month from the due date until paid in full.

2. Late payments shall bear interest at the rate of 1.5% per month from the due date until paid in full.

The penalties for late payments are intended to encourage prompt payment and ensure that the Client is
responsible for any costs incurred due to late payments.

--- Query: Summarize the key terms in this contract. ---


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



1. Service Provider agrees to provide Client with consulting services.
2. Service Provider shall use reasonable efforts to perform the Services in accordance with generally
accepted industry standards and practices.
3. Client agrees to pay Service Provider for the Services at the rates specified in Exhibit B.
4. Service Provider shall invoice Client on a monthly basis for Services performed.
5. Late payments shall bear interest at the rate of 1.5% per month from the due date until paid in full.
6. Term and Termination
6.1 This Agreement shall commence on the Effective Date and shall continue for a period of one (1) year,
unless earlier terminated as provided herein.
6.2 Either party may terminate this Agreement upon thirty (30) days written notice to the other party.
7. REFUND POLICY
7.1 If Client is dissatisfied with the Services, Client may request a refund within 14 days of service
delivery.
7.2 Refunds are issued at the sole discretion of Service Provider and will be processed wit

In [18]:
import gradio as gr

def predict(input, history=None):
    response = query_engine.query(input)
    return str(response)

gr.ChatInterface(predict).launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fdedb46b09fa1b6407.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
